In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        dtname= filename.replace(".csv", "" )
        if dtname == "train":
            train= pd.read_csv(os.path.join(dirname, filename)) 
        elif dtname == "test":    
            test= pd.read_csv(os.path.join(dirname, filename)) 

In [ ]:
train.head(10), test.head(10)

In [ ]:
train.describe().T

In [ ]:
train.info()

In [ ]:
train.R.unique()

In [ ]:
train.u_out.unique()

In [ ]:
test.head()

In [ ]:
def feature_engineering(df):
    df["u_in_sum"] = df.groupby("breath_id")["u_in"].transform("sum")
    df["u_in_cumsum"] = df.groupby("breath_id")["u_in"].cumsum()
    df["u_in_std"] = df.groupby("breath_id")["u_in"].transform("std")
    df["u_in_min"] = df.groupby("breath_id")["u_in"].transform("min")
    df["u_in_max"] = df.groupby("breath_id")["u_in"].transform("max")
    df["u_in_cumsum_reverse"] = df["u_in_sum"] - df["u_in_cumsum"]
    
    df["u_in_first"] = df.groupby("breath_id")["u_in"].transform("first")
    df["u_in_last"] = df.groupby("breath_id")["u_in"].transform("last")
    
    df["u_in_lag1"] = df.groupby("breath_id")["u_in"].shift(1)
    df["u_in_lead1"] = df.groupby("breath_id")["u_in"].shift(-1)
    df["u_in_lag1_diff"] = df["u_in"] - df["u_in_lag1"]
    df["u_in_lead1_diff"] = df["u_in"] - df["u_in_lead1"]
    
    df["u_out_sum"] = df.groupby("breath_id")["u_out"].transform("sum")
    
    df["time_passed"] = df.groupby("breath_id")["time_step"].diff()
    
    return df
    
train_1 = feature_engineering(train)
test_1 = feature_engineering(test)

In [ ]:
train_1.head()

In [ ]:
in_df = train_1[train_1["u_out"] == 0].reset_index(drop=True)
in_df.shape

!pip install lofo-importance

from lofo import Dataset, LOFOImportance, plot_importance
from sklearn.model_selection import GroupKFold

cv = list(GroupKFold(n_splits=5).split(in_df, in_df["pressure"], groups=in_df["breath_id"]))

features = ["time_step", "u_in", "R", "C",
            "u_in_sum", "u_in_cumsum", "u_in_std", "u_in_min", "u_in_max", "u_in_cumsum_reverse",
            "u_in_lead1", "u_in_lag1", "u_in_lag1_diff", "u_in_lead1_diff",
            "u_out_sum", "time_passed", "u_in_first", "u_in_last"]

ds = Dataset(in_df, target="pressure", features=features,
    feature_groups=None,
    auto_group_threshold=0.9
)
 
lofo_imp = LOFOImportance(ds, cv=cv, scoring="neg_mean_absolute_error")

importance_df = lofo_imp.get_importance()
importance_df

lofo_imp = LOFOImportance(ds, cv=cv, scoring="neg_mean_absolute_error")

importance_df = lofo_imp.get_importance()
importance_df

plot_importance(importance_df, figsize=(8, 10))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
train.dtypes.T

In [ ]:
train_1 = train_1.fillna(train_1.min())
test_1 = test_1.fillna(test_1.min())

In [ ]:
train_1.head()

In [ ]:
#drop original dependent var and id 
X = train_1.drop(['id','breath_id' , 'u_in','u_out','pressure','u_in_sum', 'u_in_std','u_in_min','u_in_max','u_in_cumsum_reverse','u_in_first',
'u_in_last','u_in_lag1','u_in_lead1','u_in_lag1_diff','u_in_lead1_diff','u_out_sum'],axis=1)
#dt.drop(['SalePrice'], axis=1)
#id	breath_id	R	C	time_step	u_in	u_out	pressure	u_in_sum	u_in_cumsum	u_in_std	u_in_min	u_in_max	u_in_cumsum_reverse	u_in_first	u_in_last	u_in_lag1	u_in_lead1	u_in_lag1_diff	u_in_lead1_diff	u_out_sum	time_passed

y=train_1.pressure
# Split train test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

In [ ]:
#1. linear regression 
from sklearn import linear_model
lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)
#r square 
print("R-Square : " ,model.score(X_test,y_test))
#rmse 
preds = model.predict(X_test)
from sklearn.metrics import mean_squared_error
print ('RMSE: ', mean_squared_error(y_test, preds))

In [ ]:
submit= pd.DataFrame()
submit['id'] = test_1.id
#select features 
test_features = test_1.drop(['id','breath_id' , 'u_in','u_out','u_in_sum', 'u_in_std','u_in_min','u_in_max','u_in_cumsum_reverse','u_in_first',
'u_in_last','u_in_lag1','u_in_lead1','u_in_lag1_diff','u_in_lead1_diff','u_out_sum'], axis=1)
preds = model.predict(test_features)
#unlog/exp the prediction  
final_preds = np.exp(preds)
print('Original preds :\n', preds[:5])
print('Final preds :\n', final_preds[:5])
submit['pressure'] = final_preds
#final submission 
filename= "venti_lr"+datetime.today().strftime('%d-%m-%Y:%H:%M:%S').replace("-","_").replace(":","_")+".csv"
submit.to_csv(filename, index=False)
print(filename ," generated")

In [ ]:
from datetime import datetime
filename= "venti_v1"+datetime.today().strftime('%Y-%m-%d-%H:%M:%S').replace("-","_").replace(":","_")+".csv"
filename

#v1 : Results are horrible though <br>
with zero - 23388693885184200000000000000000.000 :) <br>
with mean()... R-Square :  0.36168178094061076, RMSE:  41.934312966351335 <br>
with std() ... R-Square :  0.38445055999331257 RMSE:  40.43851811333169
add Code